<a href="https://colab.research.google.com/github/UiinKim/Crawling/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EA%B5%AC%EC%B6%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U datasets[audio] #datasets 라이브러리 최신화[오디오]
!pip install yt-dlp #유튜브 비디오 다운로드 기능
!pip install gradio #interactive web application
!pip install datasets
!pip install pandas
!pip install librosa
!pip install soundfile

  Using cached websockets-13.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached websockets-13.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (164 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio-client 1.3.0 requires websockets<13.0,>=10.0, but you have websockets 13.1 which is incompatible.
  Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.wh

In [4]:
import subprocess
import os
import json

def download_metadata(video_identifier, url_base='https://www.youtube.com/watch?v='):
    tmp_dir = '/tmp/crawling_data'  # 임시 디렉토리
    num_attempts = 5  # 시도 횟수
    status = False
    metadata_filename = os.path.join(tmp_dir, f"{video_identifier}_metadata.json")

    # 유튜브 메타데이터 다운로드하는 yt-dlp 명령어 실행
    command = f"""
        yt-dlp --quiet --no-warnings -j {url_base}{video_identifier}
    """.strip()

    attempts = 0
    while attempts < num_attempts:
        try:
            output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # 메타데이터 출력
    metadata = json.loads(output)
    return True, metadata

def process(video_url):
    # 유튜브 비디오 ID 추출
    video_identifier = video_url.split("v=")[-1]

    # 메타데이터 다운로드
    status, metadata = download_metadata(video_identifier)

    if not status:
        print("Failed to download metadata.")
        return None

    # 메타데이터 출력
    description = metadata.get('description', 'No description available')
    title = metadata.get('title', 'No title available')
    uploader = metadata.get('uploader', 'Unknown uploader')
    upload_date = metadata.get('upload_date', 'Unknown date')

    print(f"Title: {title}")
    print(f"Uploader: {uploader}")
    print(f"Upload Date: {upload_date}")
    print(f"Description: {description}")

# 유튜브 비디오 URL 지정
video_url = 'https://www.youtube.com/watch?v=UxPq4NjsnQE&list=PLrn-3oc_hSry7_6AHoRT6-1bpoqK5-mHt'

# 메타데이터 가져오기 및 출력
process(video_url)


Title: Mozart - Eine kleine Nachtmusik, K. 525
Uploader: Classical Music for All
Upload Date: 20210319
Description: New videos and songs everyday.
Like and subscribe to our channel: https://www.youtube.com/channel/UCnecaf9_nSWT__lhfu4EYpg

Born in Salzburg, in the Holy Roman Empire, Mozart showed prodigious ability from his earliest childhood. Already competent on keyboard and violin, he composed from the age of five and performed before European royalty. At 17, Mozart was engaged as a musician at the Salzburg court but grew restless and travelled in search of a better position. While visiting Vienna in 1781, he was dismissed from his Salzburg position. He chose to stay in Vienna, where he achieved fame but little financial security. During his final years in Vienna, he composed many of his best-known symphonies, concertos, and operas, and portions of the Requiem, which was largely unfinished at the time of his early death at the age of 35. The circumstances of his death have been much

In [ ]:
import librosa
import soundfile as sf
from datasets import Audio
from pathlib import Path

# Configuration
samples_to_load = 1000     # 데이터셋에서 로드할 오디오 파일의 수
cores = 4                 # 로딩에 사용할 프로세스의 수
sampling_rate = 44100     # 오디오 샘플링 속도
writer_batch_size = 1000  # 각 워커 당 메모리에 유지할 샘플 수
data_dir = "./music_data" # 데이터를 저장할 디렉토리

# Create directory where data will be saved
data_dir = Path(data_dir)  # 경로 객체 생성
data_dir.mkdir(exist_ok=True, parents=True)  # 디렉토리 생성, 이미 디렉토리가 있는 경우엔 오류 발생 x

# Function to process and save audio as wav files
def process(examples):
    # 각 오디오 샘플에 대한 처리 로직
    audio_data = examples['audio']['array']  # 오디오 데이터를 불러오기
    filename = examples['id']  # 각 샘플의 ID나 고유한 이름을 파일명으로 사용

    # Save audio as WAV
    save_path = data_dir / f"{filename}.wav"  # Save as .wav file in the specified directory
    sf.write(save_path, audio_data, sampling_rate)  # soundfile.write로 wav 파일 저장

    return examples  # 원래의 데이터를 반환 (필요할 경우)

# Dataset Loading & Processing Logic
def load_and_process_dataset(ds):
    # Just select some samples
    ds = ds.select(range(samples_to_load))  # 해당 숫자만큼 샘플 선택

    # Map the process function to all dataset items
    ds = ds.map(
        process,  # map을 활용하여 process 함수를 여러 프로세스로 실행
        num_proc=cores,  # 사용할 프로세스의 수
        writer_batch_size=writer_batch_size,  # 워커 당 메모리에 유지할 수
        keep_in_memory=False  # 데이터를 메모리에 유지하지 않음
    ).cast_column('audio', Audio(sampling_rate=sampling_rate))  # audio column을 Audio 형식으로 변환

    return ds

# Example: Call the function to process and save audio files
# Assuming `ds` is the dataset object containing the audio data.
# ds = load_your_dataset_here()
# load_and_process_dataset(ds)


In [ ]:
import gradio as gr

def get_example(idx):
    ex = ds[idx]
    return ex['audio']['path'], ex['caption']

gr.Interface(
    get_example,
    inputs=gr.Slider(0, len(ds) - 1, value=0, step=1),
    outputs=['audio', 'textarea'],
    allow_flagging="never",
    live=True
).launch(share=True)

In [6]:
import subprocess
import os
import json
import re

def download_metadata(video_identifier, url_base='https://www.youtube.com/watch?v='):
    tmp_dir = '/tmp/crawling_data'  # 임시 디렉토리
    num_attempts = 5  # 시도 횟수
    status = False
    metadata_filename = os.path.join(tmp_dir, f"{video_identifier}_metadata.json")

    # 유튜브 메타데이터 다운로드하는 yt-dlp 명령어 실행
    command = f"""
        yt-dlp --quiet --no-warnings -j {url_base}{video_identifier}
    """.strip()

    attempts = 0
    while attempts < num_attempts:
        try:
            output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # 메타데이터 출력
    metadata = json.loads(output)
    return True, metadata

def split_title(full_title):
    # 작곡가 이름과 제목을 구분하기 위해 '-' 문자로 분리
    match = re.split(r'\s*-\s*', full_title, maxsplit=1)
    if len(match) == 2:
        composer = match[0].strip()
        title = match[1].strip()
    else:
        composer = 'Unknown'
        title = full_title.strip()

    return composer, title

def process(video_url):
    # 유튜브 비디오 ID 추출
    video_identifier = video_url.split("v=")[-1]

    # 메타데이터 다운로드
    status, metadata = download_metadata(video_identifier)

    if not status:
        print("Failed to download metadata.")
        return None

    # 메타데이터에서 제목 추출 및 분리
    full_title = metadata.get('title', 'No title available')
    composer, title = split_title(full_title)

    uploader = metadata.get('uploader', 'Unknown uploader')
    upload_date = metadata.get('upload_date', 'Unknown date')
    description = metadata.get('description', 'No description available')
    duration = metadata.get('duration', 'Unknown duration')

    # 결과 출력
    print(f"Composer: {composer}")
    print(f"Title: {title}")
    print(f"Uploader: {uploader}")
    print(f"Upload Date: {upload_date}")
    print(f"Duration: {duration}")
    print(f"Description: {description}")

# 유튜브 비디오 URL 지정
video_url = 'https://www.youtube.com/watch?v=UxPq4NjsnQE&list=PLrn-3oc_hSry7_6AHoRT6-1bpoqK5-mHt'

# 메타데이터 가져오기 및 출력
process(video_url)


Composer: Mozart
Title: Eine kleine Nachtmusik, K. 525
Uploader: Classical Music for All
Upload Date: 20210319
Duration: 340
Description: New videos and songs everyday.
Like and subscribe to our channel: https://www.youtube.com/channel/UCnecaf9_nSWT__lhfu4EYpg

Born in Salzburg, in the Holy Roman Empire, Mozart showed prodigious ability from his earliest childhood. Already competent on keyboard and violin, he composed from the age of five and performed before European royalty. At 17, Mozart was engaged as a musician at the Salzburg court but grew restless and travelled in search of a better position. While visiting Vienna in 1781, he was dismissed from his Salzburg position. He chose to stay in Vienna, where he achieved fame but little financial security. During his final years in Vienna, he composed many of his best-known symphonies, concertos, and operas, and portions of the Requiem, which was largely unfinished at the time of his early death at the age of 35. The circumstances of hi

In [8]:
import pandas as pd
import subprocess
import json
import re

# 유튜브 메타데이터 다운로드 함수
def download_metadata(video_identifier, url_base='https://www.youtube.com/watch?v='):
    num_attempts = 5  # 시도 횟수
    status = False

    # 유튜브 메타데이터 다운로드하는 yt-dlp 명령어 실행
    command = f"""
        yt-dlp --quiet --no-warnings -j {url_base}{video_identifier}
    """.strip()

    attempts = 0
    while attempts < num_attempts:
        try:
            output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # 메타데이터 출력
    metadata = json.loads(output)
    return True, metadata

# 제목에서 작곡가와 곡 제목을 분리하는 함수
def split_title(full_title):
    match = re.split(r'\s*-\s*', full_title, maxsplit=1)
    if len(match) == 2:
        composer = match[0].strip()
        title = match[1].strip()
    else:
        composer = 'Unknown'
        title = full_title.strip()

    return composer, title

# ytid 리스트를 기반으로 데이터프레임 생성 함수
def process_videos_from_csv(csv_file_path):
    # CSV 파일에서 데이터 로드
    df = pd.read_csv(csv_file_path)

    # 결과를 저장할 빈 리스트
    data_list = []

    # 각 ytid에 대해 메타데이터 추출
    for video_identifier in df['ytid'][:10]:
        # 메타데이터 다운로드
        status, metadata = download_metadata(video_identifier)

        if not status:
            print(f"Failed to download metadata for video ID: {video_identifier}")
            continue

        # 메타데이터에서 제목 추출 및 분리
        full_title = metadata.get('title', 'No title available')
        composer, title = split_title(full_title)
        duration = metadata.get('duration', 'Unknown duration')

        # 데이터를 리스트에 추가
        data_list.append({
            "ytid": video_identifier,
            "Composer": composer,
            "Title": title,
            "Duration(s)": duration
        })

    # 리스트를 데이터프레임으로 변환
    result_df = pd.DataFrame(data_list)
    return result_df

# 예시 CSV 파일 경로
csv_file_path = '/content/musiccaps-public.csv'

# 동영상 메타데이터 처리 및 데이터프레임 생성
result_df = process_videos_from_csv(csv_file_path)

# 데이터프레임 출력
print(result_df)


          ytid                                        Composer  \
0  -0Gj8-vB1q4                                       lds music   
1  -0SdAVK79lg  Learn an Easy Country Blues Lead Guitar Lesson   
2  -0vPFx-wRRI                                           徳永ゆうき   
3  -0xzrMun0Rs                                         Unknown   
4  -1LrH01Ei1w                                         Unknown   
5  -1OlgJWehn8                                         Unknown   
6  -1UWSisR2zo                                         Unknown   
7  -3Kv4fdm7Uk                                         Unknown   
8  -4NLarMj4xU                                    Tessa Violet   
9  -4SYC2YgzL8                                  Noel Gallagher   

                                               Title  Duration(s)  
0                                       perfect love          241  
1                                              EP144         1058  
2                                    「涙そうそう～指ぱっちん編～」          136  
3

In [9]:
import os
import pandas as pd
import subprocess
import json
import re

# 유튜브 메타데이터 다운로드 함수
def download_metadata(video_identifier, url_base='https://www.youtube.com/watch?v='):
    num_attempts = 5  # 시도 횟수
    status = False

    # 유튜브 메타데이터 다운로드하는 yt-dlp 명령어 실행
    command = f"""
        yt-dlp --quiet --no-warnings -j {url_base}{video_identifier}
    """.strip()

    attempts = 0
    while attempts < num_attempts:
        try:
            output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # 메타데이터 출력
    metadata = json.loads(output)
    return True, metadata

# 제목에서 작곡가와 곡 제목을 분리하는 함수
def split_title(full_title):
    match = re.split(r'\s*-\s*', full_title, maxsplit=1)
    if len(match) == 2:
        composer = match[0].strip()
        title = match[1].strip()
    else:
        composer = 'Unknown'
        title = full_title.strip()

    return composer, title

# 유튜브 동영상을 wav 파일로 변환하는 함수
def download_video_as_wav(video_identifier, output_directory, index):
    # 출력 파일 경로
    output_file = os.path.join(output_directory, f"{index}.wav")

    # yt-dlp 명령어를 사용해 동영상을 wav로 변환
    command = f"""
        yt-dlp -x --audio-format wav --output "{output_file}" https://www.youtube.com/watch?v={video_identifier}
    """

    try:
        subprocess.run(command, shell=True, check=True)
        print(f"Downloaded {video_identifier} as {output_file}")
    except subprocess.CalledProcessError as err:
        print(f"Failed to download video {video_identifier}: {err}")

# ytid 리스트를 기반으로 데이터프레임 생성 및 wav 파일 변환 함수
def process_videos_and_save_wav(csv_file_path, output_directory):
    # CSV 파일에서 데이터 로드
    df = pd.read_csv(csv_file_path)

    # 저장할 디렉토리 생성
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # 각 ytid에 대해 메타데이터 추출 및 wav 파일로 저장
    for index, video_identifier in enumerate(df['ytid'][:10]):
        # 메타데이터 다운로드
        status, metadata = download_metadata(video_identifier)

        if not status:
            print(f"Failed to download metadata for video ID: {video_identifier}")
            continue

        # wav 파일로 변환하여 저장
        download_video_as_wav(video_identifier, output_directory, index)

# 예시 CSV 파일 경로 및 출력 디렉토리
csv_file_path = '/content/musiccaps-public.csv'
output_directory = '/content/wav_files'

# 동영상 메타데이터 처리 및 wav 파일로 저장
process_videos_and_save_wav(csv_file_path, output_directory)


Downloaded -0Gj8-vB1q4 as /content/wav_files/0.wav
Downloaded -0SdAVK79lg as /content/wav_files/1.wav
Downloaded -0vPFx-wRRI as /content/wav_files/2.wav
Downloaded -0xzrMun0Rs as /content/wav_files/3.wav
Downloaded -1LrH01Ei1w as /content/wav_files/4.wav
Downloaded -1OlgJWehn8 as /content/wav_files/5.wav
Downloaded -1UWSisR2zo as /content/wav_files/6.wav
Downloaded -3Kv4fdm7Uk as /content/wav_files/7.wav
Downloaded -4NLarMj4xU as /content/wav_files/8.wav
Downloaded -4SYC2YgzL8 as /content/wav_files/9.wav


In [10]:
import os
import pandas as pd
import subprocess
import json
import re

# 유튜브 메타데이터 다운로드 함수
def download_metadata(video_identifier, url_base='https://www.youtube.com/watch?v='):
    num_attempts = 5  # 시도 횟수
    status = False

    # 유튜브 메타데이터 다운로드하는 yt-dlp 명령어 실행
    command = f"""
        yt-dlp --quiet --no-warnings -j {url_base}{video_identifier}
    """.strip()

    attempts = 0
    while attempts < num_attempts:
        try:
            output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # 메타데이터 출력
    metadata = json.loads(output)
    return True, metadata

# 제목에서 작곡가와 곡 제목을 분리하는 함수
def split_title(full_title):
    match = re.split(r'\s*-\s*', full_title, maxsplit=1)
    if len(match) == 2:
        composer = match[0].strip()
        title = match[1].strip()
    else:
        composer = 'Unknown'
        title = full_title.strip()

    return composer, title

# 유튜브 동영상을 wav 파일로 변환하는 함수
def download_video_as_wav(video_identifier, output_directory, index):
    # 출력 파일 경로
    output_file = os.path.join(output_directory, f"{index}.wav")

    # yt-dlp 명령어를 사용해 동영상을 wav로 변환
    command = f"""
        yt-dlp -x --audio-format wav --output "{output_file}" https://www.youtube.com/watch?v={video_identifier}
    """

    try:
        subprocess.run(command, shell=True, check=True)
        print(f"Downloaded {video_identifier} as {output_file}")
    except subprocess.CalledProcessError as err:
        print(f"Failed to download video {video_identifier}: {err}")

# ytid 리스트를 기반으로 데이터프레임 생성 및 wav 파일 변환 함수
def process_videos_and_save_wav(csv_file_path, output_directory, output_csv_path):
    # CSV 파일에서 데이터 로드
    df = pd.read_csv(csv_file_path)

    # 저장할 디렉토리 생성
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # 결과를 저장할 빈 리스트
    data_list = []

    # 각 ytid에 대해 메타데이터 추출 및 wav 파일로 저장
    for index, video_identifier in enumerate(df['ytid'][:10]):
        # 메타데이터 다운로드
        status, metadata = download_metadata(video_identifier)

        if not status:
            print(f"Failed to download metadata for video ID: {video_identifier}")
            continue

        # 메타데이터에서 제목 추출 및 분리
        full_title = metadata.get('title', 'No title available')
        composer, title = split_title(full_title)
        duration = metadata.get('duration', 'Unknown duration')

        # 데이터를 리스트에 추가
        data_list.append({
            "ytid": video_identifier,
            "Composer": composer,
            "Title": title,
            "Duration(s)": duration
        })

        # wav 파일로 변환하여 저장
        download_video_as_wav(video_identifier, output_directory, index)

    # 리스트를 데이터프레임으로 변환
    result_df = pd.DataFrame(data_list)

    # 결과를 CSV 파일로 저장
    result_df.to_csv(output_csv_path, index=False)
    print(f"Metadata saved to {output_csv_path}")
    return result_df

# 예시 CSV 파일 경로 및 출력 디렉토리
csv_file_path = '/content/musiccaps-public.csv'
output_directory = '/content/wav_files'
output_csv_path = '/content/video_metadata.csv'

# 동영상 메타데이터 처리 및 wav 파일로 저장, 데이터프레임도 CSV로 저장
result_df = process_videos_and_save_wav(csv_file_path, output_directory, output_csv_path)

# 데이터프레임 출력
print(result_df)


Downloaded -0Gj8-vB1q4 as /content/wav_files/0.wav
Downloaded -0SdAVK79lg as /content/wav_files/1.wav
Downloaded -0vPFx-wRRI as /content/wav_files/2.wav
Downloaded -0xzrMun0Rs as /content/wav_files/3.wav
Downloaded -1LrH01Ei1w as /content/wav_files/4.wav
Downloaded -1OlgJWehn8 as /content/wav_files/5.wav
Downloaded -1UWSisR2zo as /content/wav_files/6.wav
Downloaded -3Kv4fdm7Uk as /content/wav_files/7.wav
Downloaded -4NLarMj4xU as /content/wav_files/8.wav
Downloaded -4SYC2YgzL8 as /content/wav_files/9.wav
Metadata saved to /content/video_metadata.csv
          ytid                                        Composer  \
0  -0Gj8-vB1q4                                       lds music   
1  -0SdAVK79lg  Learn an Easy Country Blues Lead Guitar Lesson   
2  -0vPFx-wRRI                                           徳永ゆうき   
3  -0xzrMun0Rs                                         Unknown   
4  -1LrH01Ei1w                                         Unknown   
5  -1OlgJWehn8                                  

In [ ]:
print()